## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Ilulissat,GL,69.2167,-51.1000,-9.4,45,20,3.44,few clouds
1,1,Jamestown,US,42.0970,-79.2353,62.6,39,1,8.61,clear sky
2,2,Albany,US,42.6001,-73.9662,57.0,45,76,6.17,broken clouds
3,3,Sola,VU,-13.8833,167.5500,84.2,79,75,5.75,broken clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,82.4,74,20,11.50,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 93


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
3,3,Sola,VU,-13.8833,167.5500,84.20,79,75,5.75,broken clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,20,11.50,few clouds
6,6,Huejuquilla El Alto,MX,22.6000,-103.8667,82.63,4,89,17.67,overcast clouds
10,10,Diapaga,BF,12.0708,1.7889,84.85,10,3,9.84,clear sky
12,12,Vaini,TO,-21.2000,-175.2000,82.40,83,40,9.22,scattered clouds
15,15,Georgetown,MY,5.4112,100.3354,81.00,94,40,5.75,light rain
49,49,Samarai,PG,-10.6167,150.6667,83.80,76,56,13.76,broken clouds
57,57,Russell,US,32.3502,-85.1999,84.20,54,20,9.22,few clouds
62,62,Neiafu,TO,-18.6500,-173.9833,84.20,79,40,11.50,scattered clouds
66,66,Faanui,PF,-16.4833,-151.7500,82.53,73,35,21.14,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            111
City               111
Country            111
Lat                111
Lng                111
Max Temp           111
Humidity           111
Cloudiness         111
Wind Speed         111
Current Weather    111
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Sola,VU,84.20,broken clouds,-13.8833,167.5500,
4,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,
6,Huejuquilla El Alto,MX,82.63,overcast clouds,22.6000,-103.8667,
10,Diapaga,BF,84.85,clear sky,12.0708,1.7889,
12,Vaini,TO,82.40,scattered clouds,-21.2000,-175.2000,
15,Georgetown,MY,81.00,light rain,5.4112,100.3354,
49,Samarai,PG,83.80,broken clouds,-10.6167,150.6667,
57,Russell,US,84.20,few clouds,32.3502,-85.1999,
62,Neiafu,TO,84.20,scattered clouds,-18.6500,-173.9833,
66,Faanui,PF,82.53,scattered clouds,-16.4833,-151.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}" 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df                

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Sola,VU,84.20,broken clouds,-13.8833,167.5500,Leumerus Bungalows
4,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Huejuquilla El Alto,MX,82.63,overcast clouds,22.6000,-103.8667,Hotel Huexuquillan
10,Diapaga,BF,84.85,clear sky,12.0708,1.7889,
12,Vaini,TO,82.40,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
677,Buala,SB,83.12,moderate rain,-8.1450,159.5921,Maringe Lagoon Lodge
680,Kibala,TD,89.60,light rain,9.1104,18.3463,
689,Celestun,MX,88.61,few clouds,20.8667,-90.4000,Posada Lilia
695,Poya,NC,83.57,clear sky,-21.3500,165.1500,


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# Code retrived from https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python

nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

hotel_df


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Sola,VU,84.20,broken clouds,-13.8833,167.5500,Leumerus Bungalows
4,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Huejuquilla El Alto,MX,82.63,overcast clouds,22.6000,-103.8667,Hotel Huexuquillan
12,Vaini,TO,82.40,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Georgetown,MY,81.00,light rain,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
664,Kieta,PG,82.63,light rain,-6.2167,155.6333,Uruna Bay Retreat
675,Voh,NC,85.41,clear sky,-20.9667,164.7000,Le gîte du Koniambo
677,Buala,SB,83.12,moderate rain,-8.1450,159.5921,Maringe Lagoon Lodge
689,Celestun,MX,88.61,few clouds,20.8667,-90.4000,Posada Lilia


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))